Разобранными на лекции методами список того, что можно применять к метрикам-отношениям, не ограничивается.
Относительно недавно (в 2018-м году) исследователи из Яндекса разработали классный метод анализа тестов над метриками-отношениями (прямо как у нас) вида x/y.

Идея метода заключается в следующем:

Вместо того, чтобы заталкивать в тест «поюзерные» CTR, можно сконструировать другую метрику и анализировать ее, но при этом гарантируется (в отличие от сглаженного CTR), что если тест на этой другой метрике «прокрасится» и увидит изменения, значит изменения есть и в метрике исходной (то есть в лайках на пользователя и в пользовательских CTR).

При этом метод сам по себе очень прост. Что это за метрика такая?

Считаем общий CTR в контрольной группе  𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠) 
Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠 
После чего сравним  t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠  
Метод простой, гарантируется, что при приличном размере выборки (как у нас — подойдет) можно бесплатно увеличить чувствительность вашей метрики (или, по крайней мере, не сделать хуже). Как по мне, это ОЧЕНЬ круто.

Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?
Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandahouse
import scipy.stats as stats
from read_db.CH import Getch

In [3]:
data = Getch('''
SELECT exp_group,
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views
FROM simulator_20220320.feed_actions
WHERE toDate(time) between '2022-03-15' and '2022-03-21'
GROUP BY exp_group, user_id''').df

In [4]:
ctr0 = data[data.exp_group == 0].likes.sum()/data[data.exp_group == 0].views.sum()
ctr1 = data[data.exp_group == 1].likes.sum()/data[data.exp_group == 0].views.sum()

group0 = data[data.exp_group == 0].copy()
group0['lin_ctr'] = data.apply(
    lambda x: x['likes'] - ctr0*x['views'], axis=1)

group1 = data[data.exp_group == 1].copy()
group1['lin_ctr'] = data.apply(
    lambda x: x['likes'] - ctr1*x['views'], axis=1)

group2 = data[data.exp_group == 2].copy()
group2['lin_ctr'] = data.apply(
    lambda x: x['likes'] - ctr1*x['views'], axis=1)

group3 = data[data.exp_group == 3].copy()
group3['lin_ctr'] = data.apply(
    lambda x: x['likes'] - ctr0*x['views'], axis=1)

Группы 1 и 2:

In [6]:
def get_smoothed_ctr(user_likes, user_views, global_ctr, alpha=5):
    smoothed_ctr = (user_likes + alpha * global_ctr)/(user_views + alpha)
    return smoothed_ctr

global_ctr_1 = data[data.exp_group == 1].likes.sum()/data[data.exp_group == 1].views.sum()
global_ctr_2 = data[data.exp_group == 2].likes.sum()/data[data.exp_group == 2].views.sum()

group01 = data[data.exp_group == 1].copy()
group01['smoothed_ctr'] = data.apply(
    lambda x: get_smoothed_ctr(x['likes'], x['views'], global_ctr_1), axis=1)

group02 = data[data.exp_group == 2].copy()
group02['smoothed_ctr'] = data.apply(
    lambda x: get_smoothed_ctr(x['likes'], x['views'], global_ctr_2), axis=1)

stats.ttest_ind(group01.smoothed_ctr,
                group02.smoothed_ctr,
                equal_var=False)

Ttest_indResult(statistic=2.2841320431616983, pvalue=0.0223769815558559)

In [7]:
stats.ttest_ind(group1.lin_ctr,
                group2.lin_ctr,
                equal_var=False)

Ttest_indResult(statistic=6.1314288874704825, pvalue=8.930310308522553e-10)

Группы 0 и 3:

In [8]:
global_ctr_0 = data[data.exp_group == 0].likes.sum()/data[data.exp_group == 0].views.sum()
global_ctr_3 = data[data.exp_group == 3].likes.sum()/data[data.exp_group == 3].views.sum()

group00 = data[data.exp_group == 0].copy()
group00['smoothed_ctr'] = data.apply(
    lambda x: get_smoothed_ctr(x['likes'], x['views'], global_ctr_0), axis=1)

group03 = data[data.exp_group == 3].copy()
group03['smoothed_ctr'] = data.apply(
    lambda x: get_smoothed_ctr(x['likes'], x['views'], global_ctr_3), axis=1)

stats.ttest_ind(group00.smoothed_ctr,
                group03.smoothed_ctr,
                equal_var=False)

Ttest_indResult(statistic=-16.34071506341663, pvalue=1.2314229288207137e-59)

In [9]:
stats.ttest_ind(group0.lin_ctr,
                group3.lin_ctr,
                equal_var=False)

Ttest_indResult(statistic=-15.214995460903827, pvalue=5.4914249479690016e-52)

Действительно, этот тест более чувствительный по сравнению с t-тестом по сглаженному CTR и снижает значение p-value на несколько порядков. В случае групп 0 и 3 p-value в любом случае был очень небольшим, но в случае групп 1 и 2 разница более заметна.